In [0]:
from pyspark.sql.functions import col, count
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [0]:
configs={"fs.azure.account.auth.type":"OAuth",
"fs.azure.account.oauth.provider.type":"org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
# Application (client) ID / Service Prinicipal Id
"fs.azure.account.oauth2.client.id":"e25ebcf7-95b4-40d7-bd5b-36128bae2693",
# Secret ID
"fs.azure.account.oauth2.client.secret":dbutils.secrets.get(scope="paris-olympic-scope",key="paris-olympic-app-secretkey-in-scope"),
#Directory (tenant) ID of Service Principal
"fs.azure.account.oauth2.client.endpoint":"https://login.microsoftonline.com/f4690ab4-a188-40aa-8bd8-b06bbd1cef70/oauth2/token"}

In [0]:
dbutils.fs.mount(
    source="abfss://paris-olympic-data@parisolympic.dfs.core.windows.net",  #container@storageaccount
    mount_point="/mnt/parisolympic",
    extra_configs=configs
)

True

In [0]:
%fs ls "/mnt/parisolympic"

path,name,size,modificationTime
dbfs:/mnt/parisolympic/raw-data/,raw-data/,0,1729229922000
dbfs:/mnt/parisolympic/transformed-data/,transformed-data/,0,1729229935000


In [0]:
dbutils.fs.ls("/mnt/parisolympic")

[FileInfo(path='dbfs:/mnt/parisolympic/raw-data/', name='raw-data/', size=0, modificationTime=1729229922000),
 FileInfo(path='dbfs:/mnt/parisolympic/transformed-data/', name='transformed-data/', size=0, modificationTime=1729229935000)]

In [0]:
spark

In [0]:
athletes = spark.read.format("csv").option("header","true").load("/mnt/parisolympic/raw-data/athletes.csv")

coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/parisolympic/raw-data/coaches.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/parisolympic/raw-data/medals.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/parisolympic/raw-data/teams.csv")


In [0]:
athletes.head(5)

[Row(code='1532872', current='True', name='ALEKSANYAN Artur', name_short='ALEKSANYAN A', name_tv='Artur ALEKSANYAN', gender='Male', function='Athlete', country_code='ARM', country='Armenia', country_long='Armenia', nationality='Armenia', nationality_long='Armenia', nationality_code='ARM', height='0.0', weight='0.0', disciplines="['Wrestling']", events='"[""Men\'s Greco-Roman 97kg""]"', birth_date='1991-10-21', birth_place='GYUMRI', birth_country='Armenia', residence_place='GYUMRI', residence_country='Armenia', nickname='White Bear', hobbies='Playing and watching football', occupation='Athlete', education='Graduated from Shirak State University (Gyumri, ARM)', family='Father, Gevorg Aleksanyan', lang='Armenian, English, Russian', coach='Gevorg Aleksanyan (ARM), father', reason='He followed his father and his uncle into the sport', hero='Footballer Zinedine Zidane (FRA), World Cup winner (1998) and European champion (2000) with France, won the Champions League as a player and three times

In [0]:
athletes.printSchema()

In [0]:
athletes = athletes.withColumn("code", col("code").cast(IntegerType())) \
                   .withColumn("height", col("height").cast("integer")) \
                   .withColumn("weight", col("weight").cast(IntegerType()))

In [0]:
#Find the top countries with highesht number of gold medals

top_gold_medal_countries = medals.filter(col("medal_type")=="Gold Medal") \
                                 .groupBy("country").agg(count("medal_type").alias("gold_medal_count")) \
                                 .orderBy("gold_medal_count",ascending=False).show()

+-------------+----------------+
|      country|gold_medal_count|
+-------------+----------------+
|        China|              40|
|United States|              40|
|        Japan|              20|
|    Australia|              18|
|       France|              16|
|  Netherlands|              15|
|Great Britain|              14|
|        Korea|              13|
|      Germany|              12|
|        Italy|              12|
|  New Zealand|              10|
|       Canada|               9|
|   Uzbekistan|               8|
|      Hungary|               6|
|        Spain|               5|
|       Sweden|               4|
|       Norway|               4|
|        Kenya|               4|
|      Ireland|               4|
|      Czechia|               3|
+-------------+----------------+
only showing top 20 rows



In [0]:
#Find the top countries with highesht number of silver medals

top_silver_medal_countries = medals.filter(col("medal_type").contains("Silver")) \
                                 .groupBy("country").agg(count("medal_type").alias("silver_medal_count")) \
                                 .orderBy("silver_medal_count",ascending=False).show()

+-------------+------------------+
|      country|silver_medal_count|
+-------------+------------------+
|United States|                44|
|        China|                27|
|       France|                26|
|Great Britain|                22|
|    Australia|                19|
|        Italy|                13|
|      Germany|                13|
|        Japan|                12|
|        Korea|                 9|
|      Hungary|                 7|
|       Brazil|                 7|
|       Canada|                 7|
|  New Zealand|                 7|
|  Netherlands|                 7|
|      IR Iran|                 6|
|       Israel|                 5|
|      Ukraine|                 5|
|       Sweden|                 4|
|       Poland|                 4|
|        Spain|                 4|
+-------------+------------------+
only showing top 20 rows



In [0]:
athletes.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/parisolympic/transformed-data/athletes")

In [0]:
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/parisolympic/transformed-data/coaches")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/parisolympic/transformed-data/medals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/parisolympic/transformed-data/teams")